# Práctica 5

date: 10/10/2023  
File: Chehade_practica5.ipynb
Author : Pablo Naim Chehade   
Email: pablo.chehade.villalba@gmail.com  
GitHub: https://github.com/Lupama2  

## Ejercicio 1

In [ ]:
#Import libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

#Hago los gráficos interactivos
%matplotlib ipympl

#Fuente y tamaño de los caracteres en los gráficos
font = {'family' : 'Arial',
        'weight' : 'normal',
        'size'   : 17}
matplotlib.rc('font', **font)

In [ ]:
#Def correlation matrix
C = 

C_12 = 



In [ ]:
def ejemplo():


    pass

def inicializacion_pesos():

    pass

def actualizacion_pesos(w):


    pass

def train():

    pass